In [1]:
# Dependencies
import requests
import json
import pandas as pd

# Google developer API key
from config import gkey

In [2]:
busisiest_airports_csv = "busiestAirports.csv"

In [3]:
busisiest_airports_df = pd.read_csv(busisiest_airports_csv, encoding="latin1", thousands = ',')

In [4]:
busisiest_airports_df.head()

,Unnamed: 0,rank,year,airport,code (iata/icao),location,country,total passengers
0,1,1,2016,HartsfieldJackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,104171935
1,2,2,2016,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,94393454
2,3,3,2016,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,83654250
3,4,4,2016,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,80921527
4,5,5,2016,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,79699762


In [5]:
busisiest_airports_df.columns

Index(['Unnamed: 0', 'rank', 'year', 'airport', 'code\r\n(iata/icao)',
       'location', 'country', 'total\r\npassengers'],
      dtype='object')

In [6]:
busisiest_airports_df_rename = \
busisiest_airports_df.rename(columns={'code\r\n(iata/icao)' : 'code (iata/icao)', 'total\r\npassengers': 'total passengers'})

In [7]:
busisiest_airports_df_rename.columns

Index(['Unnamed: 0', 'rank', 'year', 'airport', 'code (iata/icao)', 'location',
       'country', 'total passengers'],
      dtype='object')

In [8]:
busisiest_airports_df_rename['airport'] = busisiest_airports_df_rename['airport'].str.replace(r'[^\x00-\x7f]', ' ')

In [9]:
busisiest_airports_df_rename.head()

,Unnamed: 0,rank,year,airport,code (iata/icao),location,country,total passengers
0,1,1,2016,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,104171935
1,2,2,2016,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,94393454
2,3,3,2016,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,83654250
3,4,4,2016,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,80921527
4,5,5,2016,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,79699762


In [10]:
airports_list = busisiest_airports_df_rename['airport']
airports_list

0      Hartsfield Jackson Atlanta International Airport
1                 Beijing Capital International Airport
2                           Dubai International Airport
3                     Los Angeles International Airport
4                           Tokyo International Airport
5                          O'Hare International Airport
6                                      Heathrow Airport
7                       Hong Kong International Airport
8                 Shanghai Pudong International Airport
9                             Charles de Gaulle Airport
10              Dallas/Fort Worth International Airport
11                           Amsterdam Airport Schiphol
12                                    Frankfurt Airport
13                             Istanbul Atat rk Airport
14               Guangzhou Baiyun International Airport
15                John F. Kennedy International Airport
16                             Singapore Changi Airport
17                         Denver International 

In [11]:
# Tester for my target airport
target_airport = "st. louis lambert airport"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_airport, gkey) 

# Run a request to endpoint and convert result to json
airport_geo_data = requests.get(target_url).json()

# Print the json (pretty printed)
print(json.dumps(airport_geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "St. Louis Lambert International Airport",
                    "short_name": "St. Louis Lambert International Airport",
                    "types": [
                        "airport",
                        "establishment",
                        "point_of_interest",
                        "political"
                    ]
                },
                {
                    "long_name": "10701",
                    "short_name": "10701",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "Lambert International Boulevard",
                    "short_name": "Lambert International Blvd",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "St. Louis",
  

In [14]:
counter = 0
year_counter = 16

airport_full_address_list = []
airport_lat_list = []
airport_lng_list = []


for airport in airports_list:
    
    try:
       # Tester for my target airport
        target_airport = airport

        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
            'address={0}&key={1}').format(target_airport, gkey) 

        # Run a request to endpoint and convert result to json
        airport_geo_data = requests.get(target_url).json()

        airport_name = airport_geo_data["results"][0]['address_components'][0]['long_name']
        airport_full_address = airport_geo_data["results"][0]["formatted_address"]
        airport_lat = airport_geo_data["results"][0]["geometry"]['location']['lat']
        airport_lng = airport_geo_data["results"][0]["geometry"]['location']['lng']
        
        
        airport_full_address_list.append(airport_full_address)
        airport_lat_list.append(airport_lat)
        airport_lng_list.append(airport_lng)
        
        
        counter += 1
        
        if counter >= 50 and (counter % 51) == 0:
            year_counter -= 1
            counter = 1
            
            
        print(f'\nRetriveing 20{year_counter} data for {airport_name}.  \nAirport retrieval {counter} complete!!')
    
    except IndexError:
        
        airport_full_address_list.append('not found')
        airport_lat_list.append('not found')
        airport_lng_list.append('not found')
        
        counter += 1
        print('\n ERROR retriveing 20{year_counter} airport number {counter}... Skipping...')
        
        


Retriveing 2016 data for Hartsfield-Jackson Atlanta International Airport.  
Airport retrieval 1 complete!!

Retriveing 2016 data for Beijing Capital International Airport.  
Airport retrieval 2 complete!!

Retriveing 2016 data for Dubai International Airport.  
Airport retrieval 3 complete!!

Retriveing 2016 data for Los Angeles International Airport.  
Airport retrieval 4 complete!!

Retriveing 2016 data for Haneda Airport.  
Airport retrieval 5 complete!!

Retriveing 2016 data for O'Hare International Airport.  
Airport retrieval 6 complete!!

Retriveing 2016 data for Heathrow Airport.  
Airport retrieval 7 complete!!

Retriveing 2016 data for Hong Kong International Airport.  
Airport retrieval 8 complete!!

Retriveing 2016 data for Shanghai Pudong International Airport.  
Airport retrieval 9 complete!!

Retriveing 2016 data for Charles de Gaulle Airport.  
Airport retrieval 10 complete!!

Retriveing 2016 data for Dallas/Fort Worth International Airport.  
Airport retrieval 11 com


Retriveing 2015 data for Sao Paulo International Airport.  
Airport retrieval 41 complete!!

Retriveing 2015 data for Shanghai Hongqiao International Airport.  
Airport retrieval 42 complete!!

Retriveing 2015 data for Orlando International Airport.  
Airport retrieval 43 complete!!

Retriveing 2015 data for Taiwan Taoyuan International Airport.  
Airport retrieval 44 complete!!

Retriveing 2015 data for Mexico City International Airport.  
Airport retrieval 45 complete!!

Retriveing 2015 data for Airport East Road.  
Airport retrieval 46 complete!!

Retriveing 2015 data for Newark Liberty International Airport.  
Airport retrieval 47 complete!!

Retriveing 2015 data for Narita International Airport.  
Airport retrieval 48 complete!!

Retriveing 2015 data for Ninoy Aquino International Airport.  
Airport retrieval 49 complete!!

Retriveing 2015 data for Minneapolis−Saint Paul International Airport.  
Airport retrieval 50 complete!!

Retriveing 2014 data for Hartsfield-Jackson Atlanta 


Retriveing 2013 data for Munich International Airport.  
Airport retrieval 30 complete!!

Retriveing 2013 data for Sydney Airport.  
Airport retrieval 31 complete!!

Retriveing 2013 data for Indira Gandhi International Airport.  
Airport retrieval 32 complete!!

Retriveing 2013 data for Sao Paulo International Airport.  
Airport retrieval 33 complete!!

Retriveing 2013 data for Leonardo da Vinci International Airport.  
Airport retrieval 34 complete!!

Retriveing 2013 data for Toronto Pearson International Airport.  
Airport retrieval 35 complete!!

Retriveing 2013 data for Shanghai Hongqiao International Airport.  
Airport retrieval 36 complete!!

Retriveing 2013 data for Gatwick Airport.  
Airport retrieval 37 complete!!

Retriveing 2013 data for Narita International Airport.  
Airport retrieval 38 complete!!

Retriveing 2013 data for Josep Tarradellas Barcelona-El Prat Airport.  
Airport retrieval 39 complete!!

Retriveing 2013 data for Newark Liberty International Airport.  
Airpo


Retriveing 2011 data for Guangzhou Baiyun International Airport.  
Airport retrieval 19 complete!!

Retriveing 2011 data for Shanghai Pudong International Airport.  
Airport retrieval 20 complete!!

Retriveing 2011 data for San Francisco International Airport.  
Airport retrieval 21 complete!!

Retriveing 2011 data for Phoenix Sky Harbor International Airport.  
Airport retrieval 22 complete!!

Retriveing 2011 data for 5757.  
Airport retrieval 23 complete!!

Retriveing 2011 data for George Bush Intercontinental Airport.  
Airport retrieval 24 complete!!

Retriveing 2011 data for Charlotte Douglas International Airport.  
Airport retrieval 25 complete!!

Retriveing 2011 data for Miami International Airport.  
Airport retrieval 26 complete!!

Retriveing 2011 data for Munich International Airport.  
Airport retrieval 27 complete!!

Retriveing 2011 data for Kuala Lumpur International Airport.  
Airport retrieval 28 complete!!

Retriveing 2011 data for Leonardo da Vinci International Airp

In [18]:
airport_full_address_list[:6]

['Hartsfield-Jackson Atlanta International Airport (ATL), 6000 N Terminal Pkwy, Atlanta, GA 30320, USA',
 'Beijing Capital International Airport (PEK), Shunyi, Beijing Shi, China',
 'Dubai International Airport (DXB) - Dubai - United Arab Emirates',
 'Los Angeles International Airport (LAX), 1 World Way, Los Angeles, CA 90045, USA',
 'Haneda Airport (HND), Hanedakuko, Ota City, Tokyo 144-0041, Japan',
 "O'Hare International Airport (ORD), 10000 W O'Hare Ave, Chicago, IL 60666, USA"]

In [19]:
len(airport_full_address_list)

350

In [20]:
airport_lat_list[:6]

[33.6407282, 40.0798573, 25.2531745, 33.9415889, 35.5493932, 41.9741625]

In [21]:
len(airport_lat_list)

350

In [22]:
airport_lng_list[:6]

[-84.4277001, 116.6031121, 55.3656728, -118.40853, 139.7798386, -87.9073214]

In [23]:
len(airport_lng_list)

350

In [40]:
lat_lng_list = list(map(list, zip(airport_lat_list, airport_lng_list)))

lat_lng_list

[[33.6407282, -84.4277001],
 [40.0798573, 116.6031121],
 [25.2531745, 55.3656728],
 [33.9415889, -118.40853],
 [35.5493932, 139.7798386],
 [41.9741625, -87.9073214],
 [51.4700223, -0.4542955],
 [22.308047, 113.9184808],
 [31.1443439, 121.808273],
 [49.0096906, 2.5479245],
 [32.8998091, -97.0403352],
 [52.3105386, 4.7682744],
 [50.0379326, 8.562151799999999],
 [40.9829888, 28.8104425],
 [23.3959079, 113.3079699],
 [40.6413111, -73.77813909999999],
 [1.3644202, 103.9915308],
 [39.8560963, -104.6737376],
 [37.4601908, 126.4406957],
 [13.6899991, 100.7501124],
 [28.5561624, 77.0999578],
 [-6.1275282, 106.6537058],
 [37.6213129, -122.3789554],
 [2.7455962, 101.7071602],
 [40.4983322, -3.5675982],
 [36.0848353, -115.1505854],
 [30.567456, 103.9492936],
 [47.4502499, -122.3088165],
 [19.0895595, 72.8656144],
 [25.7958723, -80.2870509],
 [35.2144026, -80.9473146],
 [43.67771760000001, -79.6248197],
 [41.297445, 2.0832941],
 [33.4372686, -112.0077881],
 [51.1536621, -0.1820629],
 [25.0796514, 1

In [29]:
busisiest_airports_df_rename['address'] = airport_full_address_list

In [30]:
busisiest_airports_df_rename['latitude'] = airport_lat_list

In [31]:
busisiest_airports_df_rename['longitude'] = airport_lng_list

In [42]:
busisiest_airports_df_rename['lat lng'] = lat_lng_list

In [43]:
busisiest_airports_df_rename.head()

,Unnamed: 0,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
0,1,1,2016,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,104171935,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
1,2,2,2016,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,94393454,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
2,3,3,2016,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,83654250,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
3,4,4,2016,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,80921527,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
4,5,5,2016,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,79699762,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"


In [44]:
del busisiest_airports_df_rename['Unnamed: 0']

In [88]:
busisiest_airports_df_rename.head()

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
0,1,2016,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,104171935,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
1,2,2016,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,94393454,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
2,3,2016,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,83654250,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
3,4,2016,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,80921527,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
4,5,2016,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,79699762,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"


In [95]:
busisiest_airports_df_rename.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\all_years_airport_data.csv', index = None)


In [96]:
twenty_ten_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2010]

In [97]:
twenty_ten_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
300,1,2010,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,89331622,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
301,2,2010,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,73948113,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
302,3,2010,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,66774738,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
303,4,2010,London Heathrow Airport,LHR/EGLL,"Hillingdon, Greater London, England",United Kingdom,65884143,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
304,5,2010,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,64211074,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
305,6,2010,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,59070127,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
306,7,2010,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Val d'Oise, Île-de-France",France,58167062,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
307,8,2010,Dallas-Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,56906610,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"
308,9,2010,Frankfurt Airport,FRA/EDDF,"Frankfurt, Hesse",Germany,53009221,"Frankfurt Airport (FRA), 60547 Frankfurt am Ma...",50.0379,8.56215,"[50.0379326, 8.562151799999999]"
309,10,2010,Denver International Airport,DEN/KDEN,"Denver, Colorado",United States,52209377,"Denver International Airport (DEN), 8500 Peña ...",39.8561,-104.674,"[39.8560963, -104.6737376]"


In [98]:
twenty_ten_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2010_airport_data.csv', index = None)


In [99]:
twenty_eleven_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2011]

In [100]:
twenty_eleven_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
250,1,2011,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,92389023,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
251,2,2011,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,78675058,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
252,3,2011,London Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,69433565,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
253,4,2011,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,66701241,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
254,5,2011,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,62584826,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
255,6,2011,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,61862052,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
256,7,2011,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Île-de-France",France,60970551,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
257,8,2011,Dallas-Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,57832495,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"
258,9,2011,Frankfurt Airport,FRA/EDDF,"Frankfurt, Hesse",Germany,56436255,"Frankfurt Airport (FRA), 60547 Frankfurt am Ma...",50.0379,8.56215,"[50.0379326, 8.562151799999999]"
259,10,2011,Hong Kong International Airport,HKG/VHHH,"Chek Lap Kok, Hong Kong",China,53328613,"Hong Kong International Airport (HKG), 1 Sky P...",22.308,113.918,"[22.308047, 113.9184808]"


In [101]:
twenty_eleven_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2011_airport_data.csv', index = None)


In [102]:
twenty_twelve_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2012]

In [103]:
twenty_twelve_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
200,1,2012,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,94956643,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
201,2,2012,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,81929359,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
202,3,2012,London Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,70037417,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
203,4,2012,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,67788722,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
204,5,2012,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,66834931,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
205,6,2012,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,63688121,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
206,7,2012,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Île-de-France",France,61611934,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
207,8,2012,Dallas-Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,58591842,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"
208,9,2012,Soekarno-Hatta International Airport,CGK/WIII,"Cengkareng, Tangerang, Banten",Indonesia,57730732,"Pajang, Benda, Tangerang City, Banten 15126, I...",-6.12665,106.655,"[-6.126647800000001, 106.6554719]"
209,10,2012,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,57684550,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"


In [104]:
twenty_twelve_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2012_airport_data.csv', index = None)


In [105]:
twenty_thirteen_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2013]

In [106]:
twenty_thirteen_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
150,1,2013,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,94430785,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
151,2,2013,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,83712355,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
152,3,2013,Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,72368030,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
153,4,2013,Tokyo Haneda Airport,HND/RJTT,"Ota, Tokyo",Japan,68906636,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
154,5,2013,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,66883271,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
155,6,2013,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,66702252,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
156,7,2013,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,66431533,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
157,8,2013,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Île-de-France",France,62052917,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
158,9,2013,Dallas/Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,60436266,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"
159,10,2013,Soekarno-Hatta International Airport,CGK/WIII,"Cengkareng, Tangerang, Banten",Indonesia,59701543,"Pajang, Benda, Tangerang City, Banten 15126, I...",-6.12665,106.655,"[-6.126647800000001, 106.6554719]"


In [107]:
twenty_thirteen_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2013_airport_data.csv', index = None)


In [108]:
twenty_fourteen_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2014]

In [109]:
twenty_fourteen_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
100,1,2014,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,96178899,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
101,2,2014,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,86128270,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
102,3,2014,London Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,73408489,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
103,4,2014,Tokyo Haneda Airport,HND/RJTT,"Ota, Tokyo",Japan,72826565,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
104,5,2014,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,70663265,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
105,6,2014,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,70475636,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
106,7,2014,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,69999010,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
107,8,2014,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Île-de-France",France,63813756,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
108,9,2014,Dallas/Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,63554402,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"
109,10,2014,Hong Kong International Airport,HKG/VHHH,"Chek Lap Kok, Hong Kong",China,63121786,"Hong Kong International Airport (HKG), 1 Sky P...",22.308,113.918,"[22.308047, 113.9184808]"


In [110]:
twenty_fourteen_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2014_airport_data.csv', index = None)


In [111]:
twenty_fifteen_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2015]

In [112]:
twenty_fifteen_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
50,1,2015,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,101491106,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
51,2,2015,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,89938628,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
52,3,2015,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,78014841,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
53,4,2015,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,76942493,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
54,5,2015,Tokyo Haneda Airport,HND/RJTT,"Ota, Tokyo",Japan,75316718,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
55,6,2015,London Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,74989795,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
56,7,2015,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,74937004,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
57,8,2015,Hong Kong International Airport,HKG/VHHH,"Chek Lap Kok, Hong Kong",China,68283407,"Hong Kong International Airport (HKG), 1 Sky P...",22.308,113.918,"[22.308047, 113.9184808]"
58,9,2015,Paris-Charles de Gaulle Airport,CDG/LFPG,"Roissy-en-France, Île-de-France",France,65766986,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"
59,10,2015,Dallas/Fort Worth International Airport,DFW/KDFW,"Dallas-Fort Worth, Texas",United States,65512163,"DFW International Airport (DFW), 2400 Aviation...",32.8998,-97.0403,"[32.8998091, -97.0403352]"


In [113]:
twenty_fifteen_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2015_airport_data.csv', index = None)


In [114]:
twenty_sixteen_busisiest_airports_df = busisiest_airports_df_rename[busisiest_airports_df_rename['year'] == 2016]

In [115]:
twenty_sixteen_busisiest_airports_df

,rank,year,airport,code (iata/icao),location,country,total passengers,address,latitude,longitude,lat lng
0,1,2016,Hartsfield Jackson Atlanta International Airport,ATL/KATL,"Atlanta, Georgia",United States,104171935,Hartsfield-Jackson Atlanta International Airpo...,33.6407,-84.4277,"[33.6407282, -84.4277001]"
1,2,2016,Beijing Capital International Airport,PEK/ZBAA,"Chaoyang-Shunyi, Beijing",China,94393454,"Beijing Capital International Airport (PEK), S...",40.0799,116.603,"[40.0798573, 116.6031121]"
2,3,2016,Dubai International Airport,DXB/OMDB,"Garhoud, Dubai",United Arab Emirates,83654250,Dubai International Airport (DXB) - Dubai - Un...,25.2532,55.3657,"[25.2531745, 55.3656728]"
3,4,2016,Los Angeles International Airport,LAX/KLAX,"Los Angeles, California",United States,80921527,"Los Angeles International Airport (LAX), 1 Wor...",33.9416,-118.409,"[33.9415889, -118.40853]"
4,5,2016,Tokyo International Airport,HND/RJTT,"Ota, Tokyo",Japan,79699762,"Haneda Airport (HND), Hanedakuko, Ota City, To...",35.5494,139.78,"[35.5493932, 139.7798386]"
5,6,2016,O'Hare International Airport,ORD/KORD,"Chicago, Illinois",United States,78327479,"O'Hare International Airport (ORD), 10000 W O'...",41.9742,-87.9073,"[41.9741625, -87.9073214]"
6,7,2016,Heathrow Airport,LHR/EGLL,"Hillingdon, London",United Kingdom,75715474,"Heathrow Airport (LHR), Longford TW6, UK",51.47,-0.454296,"[51.4700223, -0.4542955]"
7,8,2016,Hong Kong International Airport,HKG/VHHH,"Chek Lap Kok, Hong Kong",China,70314462,"Hong Kong International Airport (HKG), 1 Sky P...",22.308,113.918,"[22.308047, 113.9184808]"
8,9,2016,Shanghai Pudong International Airport,PVG/ZSPD,"Pudong, Shanghai",China,66002414,"Shanghai Pudong International Airport (PVG), S...",31.1443,121.808,"[31.1443439, 121.808273]"
9,10,2016,Charles de Gaulle Airport,CDG/LFPG,"Paris (Roissy-en-France), Île-de-France",France,65933145,"Charles de Gaulle Airport (CDG), 95700 Roissy-...",49.0097,2.54792,"[49.0096906, 2.5479245]"


In [116]:
twenty_sixteen_busisiest_airports_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_2\output\2016_airport_data.csv', index = None)
